In [81]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt
import math
import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [82]:
start = dt.datetime.today()-dt.timedelta(10000)
end = dt.datetime.today()
#ticker = 'NVDA'
stock = yf.Ticker("MSFT")

In [83]:
#df = yf.download(ticker, start, end)
df = stock.history(period="max")
df.tail()
#df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-06-13,245.110001,249.020004,241.529999,242.259995,46135800,0.0,0.0
2022-06-14,243.860001,245.740005,241.509995,244.490005,28651500,0.0,0.0
2022-06-15,248.309998,255.300003,246.419998,251.759995,33111700,0.0,0.0
2022-06-16,245.979996,247.419998,243.020004,244.970001,33169200,0.0,0.0
2022-06-17,244.699997,250.500000,244.029999,247.649994,42800400,0.0,0.0


In [84]:
def psar(df, iaf = 0.02, maxaf = 0.2):
    length = len(df)
    dates = list(df.index)
    high = list(df['High'])
    low = list(df['Low'])
    close = list(df['Close'])
    psar = close[0:len(close)]
    psarbull = [None] * length # Bullish signal - dot below candle
    psarbear = [None] * length # Bearish signal - dot above candle
    bull = True
    af = iaf # acceleration factor
    ep = low[0] # ep = Extreme Point
    hp = high[0] # High Point
    lp = low[0] # Low Point

    # https://www.investopedia.com/terms/p/parabolicindicator.asp - Parabolic Stop & Reverse Formula from Investopedia 
    for i in range(2,length):
        if bull:
            psar[i] = psar[i - 1] + af * (hp - psar[i - 1])
        else:
            psar[i] = psar[i - 1] + af * (lp - psar[i - 1])
        reverse = False
        if bull:
            if low[i] < psar[i]:
                bull = False
                reverse = True
                psar[i] = hp
                lp = low[i]
                af = iaf
        else:
            if high[i] > psar[i]:
                bull = True
                reverse = True
                psar[i] = lp
                hp = high[i]
                af = iaf
        if not reverse:
            if bull:
                if high[i] > hp:
                    hp = high[i]
                    af = min(af + iaf, maxaf)
                if low[i - 1] < psar[i]:
                    psar[i] = low[i - 1]
                if low[i - 2] < psar[i]:
                    psar[i] = low[i - 2]
            else:
                if low[i] < lp:
                    lp = low[i]
                    af = min(af + iaf, maxaf)
                if high[i - 1] > psar[i]:
                    psar[i] = high[i - 1]
                if high[i - 2] > psar[i]:
                    psar[i] = high[i - 2]
        if bull:
            psarbull[i] = psar[i]
        else:
            psarbear[i] = psar[i]
    return {"dates":dates, "high":high, "low":low, "close":close, "psar":psar, "psarbear":psarbear, "psarbull":psarbull}

In [85]:
startidx = 0
endidx = len(df)

result = psar(df)
dates = result['dates'][startidx:endidx]
close = result['close'][startidx:endidx]
df["PSAR"] = result['psar'][startidx:endidx]
psarbear = result['psarbear'][startidx:endidx]
psarbull = result['psarbull'][startidx:endidx]
df['Slow MA'] = df['Close'].rolling(200).mean()
df['Fast MA'] = df['Close'].rolling(50).mean()

# plt.title('PSAR (Parabolic Stop & Reverse)')
# plt.plot(dates, close, label = ticker)
# plt.plot(dates, psarbull, label = 'Buy', color = 'green')
# plt.plot(dates, psarbear, label = 'Sell', color = 'red' )
# plt.plot(df['200 MA'], label = '200 Day MA')
# plt.legend()
# plt.grid()
# plt.show()

#fig = go.Figure(data=[go.Candlestick(x=df.index,
#                open=df['Open'],
#                high=df['High'],
#                low=df['Low'],
#                close=df['Close'])])

#fig.add_trace(go.Scatter(x=dates, y=psarbull, name='buy',mode = 'markers',
#                         marker = dict(color='green', size=4)))

#fig.add_trace(go.Scatter(x=dates, y=psarbear, name='sell', mode = 'markers',
#                         marker = dict(color='red', size=4)))

#fig.add_trace(go.Scatter(x=df.index, y=df['Slow MA'], name='Slow MA',
#                         line = dict(color='orange', width=2)))

#fig.add_trace(go.Scatter(x=df.index, y=df['Fast MA'], name='fast MA',
#                          line = dict(color='Blue', width=2)))

#fig.show()

In [86]:
#import talib
df.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,PSAR,Slow MA,Fast MA
Date,,,,,,,,,,
2022-06-13,245.110001,249.020004,241.529999,242.259995,46135800,0.0,0.0,277.186802,300.350937,274.905424
2022-06-14,243.860001,245.740005,241.509995,244.490005,28651500,0.0,0.0,275.760530,300.084080,273.621203
2022-06-15,248.309998,255.300003,246.419998,251.759995,33111700,0.0,0.0,273.705498,299.834342,272.371641
2022-06-16,245.979996,247.419998,243.020004,244.970001,33169200,0.0,0.0,271.773768,299.559151,271.067889
2022-06-17,244.699997,250.500000,244.029999,247.649994,42800400,0.0,0.0,269.957941,299.297608,270.044808


In [87]:
#df['50 MA'], df['200 MA'] = talib.MA(df['Adj Close'], timeperiod=50, matype=0), talib.MA(df['Adj Close'], timeperiod=200, matype=0)
df['50 MA'] = df["Close"].rolling(50).mean()
df['200 MA'] = df["Close"].rolling(200).mean()
# df['9 MA'], df['21 MA'] = talib.MA(df['Adj Close'], timeperiod=9, matype=0), talib.MA(df['Adj Close'], timeperiod=21, matype=0)
#df['PSAR'] = real = talib.SAR(df['High'], df['Low'], acceleration=0.02, maximum=0.2)
# df['upperband'], df['middleband'], df['lowerband'] = talib.BBANDS(df['Adj Close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
df['Action'] = np.where(df['Close'] > df['200 MA'] , 1, 0) 
df['Action'] = np.where(df['Close'] < df['200 MA'], -1, df['Action'])
df['PSAR_Action'] = np.where(df['PSAR'] < df['Low'] , 1, 0) 
df['PSAR_Action'] = np.where(df['PSAR'] > df['High'], -1, df['PSAR_Action'])
df.dropna(inplace=True)
df.head(10)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,PSAR,Slow MA,Fast MA,50 MA,200 MA,Action,PSAR_Action
Date,,,,,,,,,,,,,,
1986-12-24,0.108033,0.108033,0.107487,0.107760,7027200,0.0,0.0,0.108466,0.073293,0.096106,0.096106,0.073293,1,-1
1986-12-26,0.107760,0.108578,0.107487,0.107487,3715200,0.0,0.0,0.098757,0.073525,0.096728,0.096728,0.073525,1,1
1986-12-29,0.107487,0.108578,0.103123,0.103123,41702400,0.0,0.0,0.098954,0.073724,0.097262,0.097262,0.073724,1,1
1986-12-30,0.103123,0.104759,0.102031,0.104214,25401600,0.0,0.0,0.099339,0.073923,0.097775,0.097775,0.073923,1,1
1986-12-31,0.104214,0.106942,0.104214,0.105305,23356800,0.0,0.0,0.099708,0.074136,0.098299,0.098299,0.074136,1,1
1987-01-02,0.105305,0.106396,0.103668,0.104214,12643200,0.0,0.0,0.100063,0.074349,0.098736,0.098736,0.074349,1,1
1987-01-05,0.104214,0.111306,0.103668,0.110215,48499200,0.0,0.0,0.100404,0.074600,0.099292,0.099292,0.074600,1,1
1987-01-06,0.110215,0.112398,0.109670,0.111852,40032000,0.0,0.0,0.101058,0.074867,0.099881,0.099881,0.074867,1,1
1987-01-07,0.111852,0.117854,0.111852,0.117854,60998400,0.0,0.0,0.101965,0.075173,0.100602,0.100602,0.075173,1,1


In [88]:
def signal(df):
    if df['Action'] == 1 and df['PSAR_Action'] == 1:
        return 1
    elif df['Action'] == -1 and df['PSAR_Action'] == -1:
        return -1
    else:
        return 0

In [89]:
df['signal'] = df.apply(signal, axis = 1)
df.head(10)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,PSAR,Slow MA,Fast MA,50 MA,200 MA,Action,PSAR_Action,signal
Date,,,,,,,,,,,,,,,
1986-12-24,0.108033,0.108033,0.107487,0.107760,7027200,0.0,0.0,0.108466,0.073293,0.096106,0.096106,0.073293,1,-1,0
1986-12-26,0.107760,0.108578,0.107487,0.107487,3715200,0.0,0.0,0.098757,0.073525,0.096728,0.096728,0.073525,1,1,1
1986-12-29,0.107487,0.108578,0.103123,0.103123,41702400,0.0,0.0,0.098954,0.073724,0.097262,0.097262,0.073724,1,1,1
1986-12-30,0.103123,0.104759,0.102031,0.104214,25401600,0.0,0.0,0.099339,0.073923,0.097775,0.097775,0.073923,1,1,1
1986-12-31,0.104214,0.106942,0.104214,0.105305,23356800,0.0,0.0,0.099708,0.074136,0.098299,0.098299,0.074136,1,1,1
1987-01-02,0.105305,0.106396,0.103668,0.104214,12643200,0.0,0.0,0.100063,0.074349,0.098736,0.098736,0.074349,1,1,1
1987-01-05,0.104214,0.111306,0.103668,0.110215,48499200,0.0,0.0,0.100404,0.074600,0.099292,0.099292,0.074600,1,1,1
1987-01-06,0.110215,0.112398,0.109670,0.111852,40032000,0.0,0.0,0.101058,0.074867,0.099881,0.099881,0.074867,1,1,1
1987-01-07,0.111852,0.117854,0.111852,0.117854,60998400,0.0,0.0,0.101965,0.075173,0.100602,0.100602,0.075173,1,1,1


In [90]:
pd.set_option("display.max_rows", 1000)
#df.tail(100)
# IF df.iloc[-1] == 0 THEN "Closing Position / Do Nothing"
# IF df.iloc[-1] == 1 THEN "BUY"
# IF df.iloc[-1] == -1 THEN "SELL / SHORT Sell"

In [91]:
#fig1 = go.Figure(data=[go.Candlestick(x=df.index,
#                open=df['Open'],
#                high=df['High'],
#                low=df['Low'],
#                close=df['Close'])])

#fig1.add_trace(go.Scatter(x=dates, y=df['PSAR'], name='Parabolic Stop & Reverse (PSAR Signals)', mode = 'markers',
#                         marker = dict(color='black', size=4)))

#fig1.add_trace(go.Scatter(x=df.index, y=df['200 MA'], name='Slow MA',
#                         line = dict(color='orange', width=2)))

#fig1.add_trace(go.Scatter(x=df.index, y=df['50 MA'], name='fast MA',
#                         line = dict(color='Blue', width=2)))

#fig1.show()

In [92]:
# Create our features
#X = df['Adj Close', 'High', 'Low', ]

# Create our target
#y = df['loan_status']
#X.head()

In [93]:
df["signal"].iloc[-1]

-1

In [94]:
df.iloc[-1]

Open            2.447000e+02
High            2.505000e+02
Low             2.440300e+02
Close           2.476500e+02
Volume          4.280040e+07
Dividends       0.000000e+00
Stock Splits    0.000000e+00
PSAR            2.699579e+02
Slow MA         2.992976e+02
Fast MA         2.700448e+02
50 MA           2.700448e+02
200 MA          2.992976e+02
Action         -1.000000e+00
PSAR_Action    -1.000000e+00
signal         -1.000000e+00
Name: 2022-06-17 00:00:00, dtype: float64

In [95]:
type(df.iloc[-1])

pandas.core.series.Series

In [96]:
string_frame = df.iloc[-1].to_string()

In [97]:
type(string_frame)

str